In [1]:
import pandas as pd

data = pd.read_csv('/kaggle/input/train-sounds-bird-2024/trained_all_data_sound_bird_clef_2024.csv')

sample_submission = pd.read_csv('/kaggle/input/birdclef-2024/sample_submission.csv')

In [2]:
data.head()

,filename,prediction,confidence
0,/kaggle/input/birdclef-2024/train_audio/redspu...,Bird,0.858099
1,/kaggle/input/birdclef-2024/train_audio/redspu...,Animal,0.143473
2,/kaggle/input/birdclef-2024/train_audio/redspu...,Animal,0.467475
3,/kaggle/input/birdclef-2024/train_audio/redspu...,Animal,0.662496
4,/kaggle/input/birdclef-2024/train_audio/redspu...,Silence,0.435951


In [3]:
from sklearn.preprocessing import LabelEncoder

# Encode the 'prediction' column
label_encoder = LabelEncoder()
data['encoded_prediction'] = label_encoder.fit_transform(data['prediction'])

In [4]:
# Setup X and y

X = data[['confidence']].values  # Features
y = data['encoded_prediction'].values  # Target

In [5]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13373, 1), (4458, 1), (13373,), (4458,))

In [6]:
from sklearn.preprocessing import StandardScaler

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
from sklearn.preprocessing import label_binarize
import numpy as np

# Binarize y for multi-class ROC AUC
y_bin = label_binarize(y, classes=np.unique(y))

In [8]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize arrays to store all predictions and true labels
all_predictions = []
all_true_labels = []

# Perform K-Fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_bin[train_index], y_bin[test_index]
    
    # Standardize the features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define the neural network model architecture
    def build_model(input_dim, num_classes):
        model = Sequential()
        model.add(Dense(64, input_dim=input_dim, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))  # Output layer for multi-class classification
        model.compile(optimizer='adam', loss='categorical_crossentropy')
        return model

    # Build and train the model
    model = build_model(input_dim=X_train_scaled.shape[1], num_classes=y_train.shape[1])
    
    # Early stopping callback
    early_stop = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
    
    # Train the model
    model.fit(X_train_scaled, y_train, epochs=5, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[early_stop], verbose=1)
    
    # Predict probabilities on the test set
    y_pred_proba = model.predict(X_test_scaled)
    
    # Store the predictions and true labels
    all_predictions.append(y_pred_proba)
    all_true_labels.append(y_test)

# Combine all predictions and true labels
y_pred_proba_all = np.vstack(all_predictions)
y_true_all = np.vstack(all_true_labels)

# Calculate the ROC AUC score
roc_auc = roc_auc_score(y_true_all, y_pred_proba_all, multi_class='ovr')
roc_auc

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 2.9751 - val_loss: 1.6347
Epoch 2/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.6340 - val_loss: 1.6159
Epoch 3/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.6302 - val_loss: 1.6121
Epoch 4/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.5663 - val_loss: 1.6131
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


446/446 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 2.9443 - val_loss: 1.5990
Epoch 2/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.6203 - val_loss: 1.5919
Epoch 3/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.6152 - val_loss: 1.5873
Epoch 4/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.6318 - val_loss: 1.5875
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


446/446 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 2.9094 - val_loss: 1.6032
Epoch 2/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.5994 - val_loss: 1.5762
Epoch 3/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.5958 - val_loss: 1.5774
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


446/446 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 2.9211 - val_loss: 1.5967
Epoch 2/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.6380 - val_loss: 1.5855
Epoch 3/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.6179 - val_loss: 1.5848
Epoch 4/5
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.5904 - val_loss: 1.5859
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.5962607800228993

In [9]:
import pickle

# Save the trained model to a pickle file
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [10]:
y_pred_proba_corrected = y_pred_proba[:len(sample_submission)]

In [11]:
sample_submission.head()

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,soundscape_1446779_5,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
1,soundscape_1446779_10,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
2,soundscape_1446779_15,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495


In [12]:
# Save the submission to CSV
sample_submission.to_csv('submission.csv', index=False)